## Import Libraries

In [1]:
import requests
import json
import datetime
import hashlib
from datetime import timezone
import sqlite3
import codecs
import pandas
from IPython.display import display

## Set up some global variables

In [2]:
S = 0
base_url = "https://api.smitegame.com/smiteapi.svc/"
sqliteConnection = sqlite3.connect('Smite.db')
cursor = sqliteConnection.cursor()

## Create functions to interact with the API and Pull data into database

In [3]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

def Get_ts():
    ts = datetime.datetime.now(timezone.utc)
    return ts

def get_Sig(session):
    ts = Get_ts()
    to_hash = "3880" + session + "BBE7293D6D8D4223B33D4C445FF26008" + ts.strftime('%Y%m%d%H%M%S')
    output = hashlib.md5(to_hash.encode('utf-8'))
    Sig = output.hexdigest()
    return Sig

def Get_session():
    global S
    sig = get_Sig("createsession")
    Session_ts = Get_ts()
    url = "{0}createsessionJson/3880/{1}/{2}".format(base_url, sig, Session_ts.strftime('%Y%m%d%H%M%S'))
    request = requests.get(url)
    data = request.json()
    S = S + 1
    jprint(data)
    return data["session_id"], Session_ts

def Session_stats():
    session_id, session_ts = Get_session()
    sig = get_Sig("getdataused")
    ts = Get_ts()
    url = "{0}getdatausedJson/3880/{1}/{2}/{3}".format(base_url, sig, session_id, ts.strftime('%Y%m%d%H%M%S'))
    request = requests.get(url)
    data = request.json()
    jprint(data)
    return 0

def Get_IDS(session_id, session_ts, queue, date):
    sig = get_Sig("getmatchidsbyqueue")

    ts = Get_ts()

    url = "{0}getmatchidsbyqueueJson/3880/{1}/{2}/{3}/{4}/{5}/-1".format(base_url, sig, session_id, ts.strftime('%Y%m%d%H%M%S'), queue, date)
    request = requests.get(url)
    data = request.json()
    return data

def Get_match_details(session_id, ID):
    sig = get_Sig("getmatchdetailsbatch")
    ts = Get_ts()
    url = "{0}getmatchdetailsbatchJson/3880/{1}/{2}/{3}/{4}".format(base_url, sig, session_id, ts.strftime('%Y%m%d%H%M%S'), ID)
    request = requests.get(url)
#     print(url)
#     print(request)
    data = request.json()
    return data

def Get_item_data():
    sig = get_Sig("getitems")
    ts = Get_ts()
    session_id, session_ts = Get_session()
    url = "{0}/getitemsJson/3880/{1}/{2}/{3}/1".format(base_url, sig, session_id, ts.strftime('%Y%m%d%H%M%S'))
    request = requests.get(url)
    data = request.json()
    return data

def convert_item_data(data):
    stats_list = ['Health',
                 'Physical Protection',
                 'Crowd Control Reduction',
                 'Physical Power',
                 'Attack Speed',
                 'Physical Lifesteal',
                 'Physical Penetration',
                 'Magical Power',
                 'Magical Lifesteal',
                 'Mana',
                 'Magical Penetration',
                 'Cooldown Reduction',
                 'MP5',
                 'Magical Protection',
                 'Movement Speed',
                 'Critical Strike Chance',
                 'Penetration',
                 'HP5',
                 'HP5 & MP5',
                 'Magical Protection ',
                 'Magical Power ',
                 'Magical protection',
                 ' ',
                 'Unlocks at level 10',
                 'Maximum Health',
                 'Protections',
                 'Magical power',
                 'Basic Attack Damage',
                 'Physical Critical Strike Chance',
                 'Damage Reduction',
                 'Magical Protections',
                 'Physical power']
    
    exclude_keys = ['ItemDescription']
    item = {k: data[k] for k in set(list(data.keys())) - set(exclude_keys)}
    exclude_keys = ['Menuitems']
    descriptions = {k: data["ItemDescription"][k] for k in set(list(data["ItemDescription"].keys())) - set(exclude_keys)}

    value_list = list()
    for x in stats_list:
        empty = True
        for y in data["ItemDescription"]["Menuitems"]:
            if y["Description"] == x:
                try:
                    z = int(y["Value"])
                except:
                    z = y["Value"]
                value_list.append(z)
                empty = False
        if empty:
            value_list.append(0)

    Stat_dict = dict(zip(stats_list, value_list))

    item.update(Stat_dict)
    item.update(descriptions)
    
    exclude_keys = ["Magical Protection", "Magical Protection ", "Magical Protections", "Magical protection", "Magical Power", "Magical Power ", "Magical power", "Physical Power", "Physical power"]
    Version1 = {k: item[k] for k in set(list(item.keys())) - set(exclude_keys)}

    Version2 = {
        "Magical Protections": item["Magical Protection"] + item["Magical Protection "] + item["Magical Protections"] + item["Magical protection"],
        "Magical Power": item["Magical power"] + item["Magical Power "] + item["Magical Power"],
        "Physical Power": item["Physical Power"] + item["Physical power"]
    }
    final = {}
    true_final={}

    final.update(Version1)
    final.update(Version2)

    for i in sorted(final):
        true_final[i]=final[i]

    return true_final

def get_Items_full_data():
    data = Get_item_data()

    sqlite_insert_query_raw = """
            INSERT INTO {0}
                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);
            """
    sqlite_insert_query = sqlite_insert_query_raw.format("Items")

    for x in data:
        items = convert_item_data(x)
        y = list(items.values())
        cursor.execute(sqlite_insert_query, tuple(y))

def Get_gods_data():
    sig = get_Sig("getgods")
    ts = Get_ts()
    session_id, session_ts = Get_session()
    url = "{0}/getgodsJson/3880/{1}/{2}/{3}/1".format(base_url, sig, session_id, ts.strftime('%Y%m%d%H%M%S'))
    request = requests.get(url)
    data = request.json()
    return data

def SQL_gods_data():
    output = Get_gods_data()
    
    sqlite_insert_query_raw = """
        INSERT INTO {}
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);
        """
    sqlite_insert_query = sqlite_insert_query_raw.format("Gods")

    for z in output:
        exclude_keys = ["Ability_1", "Ability_2", "Ability_3", "Ability_4", "Ability_5","abilityDescription1","abilityDescription2","abilityDescription3","abilityDescription4","abilityDescription5","basicAttack"]
        god_data = {k: z[k] for k in set(list(z.keys())) - set(exclude_keys)}
        
        final = {}
        for i in sorted(god_data):
            final[i]=god_data[i]
        
        y = list(final.values())
        cursor.execute(sqlite_insert_query, tuple(y))

def Get_gods_ability_data():
    sig = get_Sig("getgodaltabilities")
    ts = Get_ts()
    session_id, session_ts = Get_session()
    url = "{0}/getgodaltabilitiesJson/3880/{1}/{2}/{3}".format(base_url, sig, session_id, ts.strftime('%Y%m%d%H%M%S'))
    request = requests.get(url)
    data = request.json()
    return data
    
    
def Get_Queue_data(Queue_id, Queue, date, Table):
    failed_IDS = list()
    
    session_id, session_ts = Get_session()

    x = Get_IDS(session_id, session_ts, Queue_id, date)

    print(len(x), "ID count")
    print(Queue)

    All_Ids = list()
    for i in x:
        All_Ids.append(i["Match"])

    All_Ids_batched = list()
    for i in range(round(len(All_Ids) / 10)):
        All_Ids_batched.append(All_Ids[10 * i: 10 * i + 9])


    for i, x in enumerate(All_Ids_batched):
        try:
            if i % 50 == 0:
                print(i)

            ts = Get_ts()
            delta = ts - session_ts

            if delta.total_seconds() > 840:
                session_id, session_ts = Get_session()

            output = Get_match_details(session_id, ','.join(x))

            sqlite_insert_query_raw = """
            INSERT INTO {}
                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);
            """
            sqlite_insert_query = sqlite_insert_query_raw.format(Table)

            for z in output:
                y = list(z.values())
                ## this pop is to remove the entry for merged players as its a dictionary and SQL cant handle it easy
                y.pop(98)
                y.append(Queue)
                cursor.execute(sqlite_insert_query, tuple(y))
        except:
            failed_IDS.append(x)
            
            
    return failed_IDS
            
            

# API Interactions

To do any interaction run the cells below the definitiion to perform that action

### Ping the server to check API status

In [4]:
url = "https://api.smitegame.com/smiteapi.svc/pingJson"
request = requests.get(url)
print(url)
print(request)
data = request.json()

https://api.smitegame.com/smiteapi.svc/pingJson
<Response [200]>


### Check the status of daily limits

In [5]:
Session_stats()

{
    "ret_msg": "Approved",
    "session_id": "09E2930D6F4F4F098F06E4458B29FAE9",
    "timestamp": "6/30/2022 10:56:50 AM"
}
[
    {
        "Active_Sessions": 1,
        "Concurrent_Sessions": 50,
        "Request_Limit_Daily": 7500,
        "Session_Cap": 500,
        "Session_Time_Limit": 15,
        "Total_Requests_Today": 5434,
        "Total_Sessions_Today": 42,
        "ret_msg": null
    }
]


0

### Pull and insert data for all of the gods base stats

In [18]:
## pulls the data for the gods stats (make sure to re run the gods table creation before running)

SQL_gods_data()
sqliteConnection.commit()

{
    "ret_msg": "Approved",
    "session_id": "9D2834896D7242ED984A9DF1E279CEDD",
    "timestamp": "6/21/2022 1:58:16 PM"
}


### Pull all of the game data for each date in the array (not each pull is around 1500 requests from the API)

In [7]:
## pull all of the queue data for a given date (API holds data for 30 days)

for date in ["20220629"]:
    print(date)
    Get_Queue_data("450", "Joust_KnM", date, "Joust_Ranked")
    Get_Queue_data("503", "Joust_Controller", date, "Joust_Ranked")
    Get_Queue_data("451", "Conq_KnM", date, "Conq_Ranked")
    Get_Queue_data("504", "Conq_Controller", date, "Conq_Ranked")
    Get_Queue_data("440", "Duel_KnM", date, "Duel_Ranked")
    Get_Queue_data("502", "Duel_Controller", date, "Duel_Ranked")
    
sqliteConnection.commit()

20220629
{
    "ret_msg": "Approved",
    "session_id": "60A98E6F62D34BD395E30F241202C6BE",
    "timestamp": "6/30/2022 10:57:35 AM"
}
875 ID count
Joust_KnM
0
50
{
    "ret_msg": "Approved",
    "session_id": "BA5C5A0CC8C3454198FA920AEAA6FF73",
    "timestamp": "6/30/2022 11:03:10 AM"
}
2825 ID count
Joust_Controller
0
50
100
150
{
    "ret_msg": "Approved",
    "session_id": "3586DD20F1A44CF3B8EDFFD9D19F7A80",
    "timestamp": "6/30/2022 11:17:12 AM"
}
200
250
{
    "ret_msg": "Approved",
    "session_id": "3A3277857CE343ED9161F8037DEFF2CE",
    "timestamp": "6/30/2022 11:22:52 AM"
}
3000 ID count
Conq_KnM
0
50
100
150
200
{
    "ret_msg": "Approved",
    "session_id": "B591C9BC84A4456AA3FC1DE2DADE911B",
    "timestamp": "6/30/2022 11:36:53 AM"
}
250
{
    "ret_msg": "Approved",
    "session_id": "2684760BA1634C8B8A619AC00A844BDC",
    "timestamp": "6/30/2022 11:41:54 AM"
}
3697 ID count
Conq_Controller
0
50
100
150
200
{
    "ret_msg": "Approved",
    "session_id": "C6D7C66BC25945CD

### Commit to the database if not already done

In [20]:
sqliteConnection.commit()

### Check the status of the match details database

In [8]:
SQL_command = """
SELECT count(*) FROM Joust_Ranked
"""

cursor.execute(SQL_command)

print("Joust", cursor.fetchall())

SQL_command = """
SELECT count(*) FROM Conq_Ranked
"""

cursor.execute(SQL_command)

print("Conquest", cursor.fetchall())

SQL_command = """
SELECT count(*) FROM Duel_Ranked
"""

cursor.execute(SQL_command)

print("Duel", cursor.fetchall())

Joust [(1522661,)]
Conquest [(4249021,)]
Duel [(490806,)]


In [9]:
SQL_command = """
SELECT DISTINCT SUBSTRING(Entry_datetime, 0, 10) FROM Joust_Ranked
"""

cursor.execute(SQL_command)

print(cursor.fetchall())

[('4/19/2022',), ('4/20/2022',), ('4/21/2022',), ('4/22/2022',), ('4/23/2022',), ('4/24/2022',), ('4/25/2022',), ('4/26/2022',), ('4/27/2022',), ('4/28/2022',), ('4/29/2022',), ('4/30/2022',), ('5/1/2022 ',), ('5/2/2022 ',), ('5/3/2022 ',), ('5/4/2022 ',), ('5/5/2022 ',), ('5/6/2022 ',), ('5/7/2022 ',), ('5/8/2022 ',), ('5/13/2022',), ('5/10/2022',), ('5/11/2022',), ('5/12/2022',), ('5/9/2022 ',), ('5/14/2022',), ('5/15/2022',), ('5/16/2022',), ('5/17/2022',), ('5/18/2022',), ('5/19/2022',), ('5/20/2022',), ('5/21/2022',), ('5/22/2022',), ('5/23/2022',), ('5/24/2022',), ('5/25/2022',), ('5/26/2022',), ('5/27/2022',), ('5/28/2022',), ('5/29/2022',), ('5/30/2022',), ('5/31/2022',), ('6/1/2022 ',), ('6/2/2022 ',), ('6/3/2022 ',), ('6/4/2022 ',), ('6/5/2022 ',), ('6/6/2022 ',), ('6/7/2022 ',), ('6/8/2022 ',), ('6/9/2022 ',), ('6/10/2022',), ('6/11/2022',), ('6/12/2022',), ('6/13/2022',), ('6/14/2022',), ('6/15/2022',), ('6/16/2022',), ('6/17/2022',), ('6/18/2022',), ('6/19/2022',), ('6/20/

In [8]:
cursor.close()